In [ ]:
# # 날짜 간 차이 계산
# df['next_date'] = df['date'] + pd.Timedelta(days=1)
# df['date_diff'] = df['next_date'] - df['date']

# print(df)

# # 시간대 설정 및 변환
# df['date_utc'] = df['date'].dt.tz_localize('UTC')
# df['date_kst'] = df['date_utc'].dt.tz_convert('Asia/Seoul')


# # 범주형 피처 인코딩
# df = pd.get_dummies(df, columns=['destination'], drop_first=True)

# # 피처와 타겟 분리
# X = df.drop(columns=['price'])
# y = df['price']

# # 비행시간 피처 포함 모델
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("MSE with flight_duration:", mean_squared_error(y_test, y_pred))

# # 비행시간 피처 제거 모델
# X = X.drop(columns=['flight_duration'])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print("MSE without flight_duration:", mean_squared_error(y_test, y_pred))


In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# 출발일로 부터 항공권 예매 시점(20240430)과의 날짜 차이 피처 추가 예정
now = pd.to_datetime("20240430", format='%Y%m%d')
print("Timestamp:", now)

Timestamp: 2024-04-30 00:00:00


In [79]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    df = pd.read_csv('/content/drive/MyDrive/직항_머신러닝.csv')
except:
    df = pd.read_csv('./data/직항_머신러닝.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
df.describe(include='all')

,nights,departure_sdt,departure_jt,dep_week,arrival_sdt,arrival_jt,arr_week,agentcode,partition_0,partition_1,time_zone,baggagetype,total_fare
count,714792.000000,714792,714792.000000,714792.000000,714792,714792.000000,714792.000000,714792,714792,714792,714792,658137,7.147920e+05
unique,NaN,5558,NaN,NaN,6590,NaN,NaN,26,9,77,47,3,NaN
top,NaN,2024-05-03 08:45:00.000,NaN,NaN,2024-05-16 10:35:00.000,NaN,NaN,WHY002,SouthEastAsia,KIX,Asia/Tokyo,F,NaN
freq,NaN,646,NaN,NaN,579,NaN,NaN,191022,282421,88759,260126,612483,NaN
mean,3.517860,NaN,351.139806,3.956662,NaN,350.508071,4.024907,NaN,NaN,NaN,NaN,NaN,6.580351e+05
std,1.692131,NaN,265.785353,1.934085,NaN,272.773675,2.004756,NaN,NaN,NaN,NaN,NaN,6.435439e+05
min,1.000000,NaN,105.000000,1.000000,NaN,110.000000,1.000000,NaN,NaN,NaN,NaN,NaN,1.126000e+05
25%,2.000000,NaN,150.000000,2.000000,NaN,150.000000,2.000000,NaN,NaN,NaN,NaN,NaN,3.319000e+05
50%,4.000000,NaN,235.000000,4.000000,NaN,240.000000,4.000000,NaN,NaN,NaN,NaN,NaN,4.869000e+05
75%,5.000000,NaN,445.000000,6.000000,NaN,440.000000,6.000000,NaN,NaN,NaN,NaN,NaN,6.894000e+05


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714792 entries, 0 to 714791
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   nights         714792 non-null  int64 
 1   departure_sdt  714792 non-null  object
 2   departure_jt   714792 non-null  int64 
 3   dep_week       714792 non-null  int64 
 4   arrival_sdt    714792 non-null  object
 5   arrival_jt     714792 non-null  int64 
 6   arr_week       714792 non-null  int64 
 7   agentcode      714792 non-null  object
 8   partition_0    714792 non-null  object
 9   partition_1    714792 non-null  object
 10  time_zone      714792 non-null  object
 11  baggagetype    658137 non-null  object
 12  total_fare     714792 non-null  int64 
dtypes: int64(6), object(7)
memory usage: 70.9+ MB


In [82]:
# str to datetime 형식 변환
df['departure_sdt'] = pd.to_datetime(df['departure_sdt'])
df['arrival_sdt'] = pd.to_datetime(df['arrival_sdt'])

# 시간 분 추출
df['departure_hour'] = df['departure_sdt'].apply(lambda x: x.hour)
df['departure_minute'] = df['departure_sdt'].apply(lambda x: x.minute)
df['arrival_hour'] = df['arrival_sdt'].apply(lambda x: x.hour)
df['arrival_minute'] = df['arrival_sdt'].apply(lambda x: x.minute)

# 출발 도착 시 분 sin cos 변환
df['departure_hour_sin'] = np.sin(2 * np.pi * df['departure_hour'] / 24)
df['departure_hour_cos'] = np.cos(2 * np.pi * df['departure_hour'] / 24)
df['departure_minute_sin'] = np.sin(2 * np.pi * df['departure_minute'] / 60)
df['departure_minute_cos'] = np.cos(2 * np.pi * df['departure_minute'] / 60)

df['arrival_hour_sin'] = np.sin(2 * np.pi * df['arrival_hour'] / 24)
df['arrival_hour_cos'] = np.cos(2 * np.pi * df['arrival_hour'] / 24)
df['arrival_minute_sin'] = np.sin(2 * np.pi * df['arrival_minute'] / 60)
df['arrival_minute_cos'] = np.cos(2 * np.pi * df['arrival_minute'] / 60)

# 출발 도착 요일 sin cos 변환
df['dep_week_sin'] = np.sin(2 * np.pi * df['dep_week'] / 7)
df['dep_week_cos'] = np.cos(2 * np.pi * df['dep_week'] / 7)
df['arr_week_sin'] = np.sin(2 * np.pi * df['arr_week'] / 7)
df['arr_week_cos'] = np.cos(2 * np.pi * df['arr_week'] / 7)


drop_col = ['departure_sdt','arrival_sdt',
            'departure_hour','departure_minute','arrival_hour','arrival_minute',
            'dep_week', 'arr_week']
df.drop(columns=drop_col,inplace=True)

In [75]:
# departure_jt, arrival_jt 출발 복귀 비행시간
# 직항 항공권 기준 목적지 피처와 다중공산성 문제가 있다고 예상 drop 전후로 모델 학습 후 판단.
# 경유 항공권 또는 비행기 도착시 현지시간으로 변환 할 때 사용

# drop_jt = ['departure_jt', 'arrival_jt']
# df.drop(columns=drop_jt,inplace=True)

In [78]:
# agentcode

df['agentcode'].value_counts()

agentcode
WHY002    191022
INT005    132661
TOV025    126221
HDP010     36763
CTR013     33286
WEB001     29843
HAT004     21279
LTT017     20460
ONT003     19443
EDR215     16647
NIT015     15247
MOT007     14674
VGT027     11854
KRT006      9921
SKP206      8107
MRT023      6115
HDT028      5456
JTB016      5019
GTG204      4243
ATK201      3226
ACA107      1431
BGT202      1068
YEL009       659
ACI117       140
ACX111         6
ALH103         1
Name: count, dtype: int64